In [1]:
import os

In [2]:
def get_folders(location):
    return [name for name in os.listdir(location) if os.path.isdir(os.path.join(location, name))]

states = get_folders('/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/SVI2018 census tracts with death rate HepVu-5 classes')

In [3]:
# loop through each state and load the shapefile and concatenate them into one dataframe
import geopandas as gpd
import pandas as pd

gdf = pd.DataFrame()
for state in states:
    print(state)
    try:
        gdf = pd.concat([gdf, gpd.read_file(f'/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/SVI2018 census tracts with death rate HepVu-5 classes/{state}/{state}.shp')])
    except:
        print(f'Error reading {state}')
        continue


/Users/h6x/anaconda3/envs/TDA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


VT
VA
SD
SC
UT
GA
MS
MT
MO
MA
AK
KY
AL
NH
MN
MI
OK
IN
CO
CA
IA
CT
FL
WV
RI
WY
TX
PA
NC
ND
NM
NJ
ME
AR
NV
MD
KS
NE
HI
DE
AZ
NY
ID
OH
OR
IL
LA
WI
WA
TN


In [4]:
gdf.shape

(72655, 129)

In [5]:
gdf.head()

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Leng,Shape_Area,NOD_Rate,percentile,geometry
0,50,VERMONT,VT,50007,Chittenden,50007980000,"Census Tract 9800, Chittenden County, Vermont",1.141485,0,9,...,0,9,-999.0,-999.0,794,0.107289,0.000337,12.5,0,"POLYGON ((-73.16975 44.48154, -73.16620 44.484..."
1,50,VERMONT,VT,50001,Addison,50001960100,"Census Tract 9601, Addison County, Vermont",81.549039,3928,202,...,214,67,5.5,1.7,1619,0.743236,0.023958,12.9,0,"POLYGON ((-73.19680 44.26663, -73.16182 44.271..."
2,50,VERMONT,VT,50001,Addison,50001960200,"Census Tract 9602, Addison County, Vermont",47.443526,2736,23,...,151,75,5.5,2.7,1993,0.650662,0.017827,12.9,0,"POLYGON ((-73.39963 44.15533, -73.39873 44.162..."
3,50,VERMONT,VT,50001,Addison,50001960300,"Census Tract 9603, Addison County, Vermont",2.476118,2612,19,...,59,34,2.3,1.3,3360,0.109137,0.000744,12.9,0,"POLYGON ((-73.27275 44.17683, -73.24845 44.179..."
4,50,VERMONT,VT,50001,Addison,50001960400,"Census Tract 9604, Addison County, Vermont",124.237386,5083,179,...,183,41,3.6,0.8,2554,0.980535,0.040436,12.9,0,"POLYGON ((-73.43774 44.04501, -73.43199 44.063..."


In [14]:
# get the unque STCNTY values and their relevent Percentile column values
A = gdf[['STCNTY', 'percentile','NOD_Rate']].drop_duplicates().sort_values(by=['STCNTY'])


In [15]:
A.shape

(3140, 3)

In [16]:
# Calculate the min and max of the column
min_value = A['NOD_Rate'].min()
max_value = A['NOD_Rate'].max()

In [17]:
min_value

-9.0

In [18]:
max_value

114.4

In [19]:
# get the values where A['NOD_Rate'] less than 0
A[A['NOD_Rate'] < 0]

,STCNTY,percentile,NOD_Rate
114,02105,0,-9.0
87,02195,0,-9.0
88,02198,1,-9.0
129,02230,0,-9.0
93,02275,0,-9.0
0,08001,0,-9.0
1,08014,0,-9.0
9,08059,0,-9.0
13,08123,0,-9.0


In [20]:
# Apply the normalization formula
A['NOD'] = (A['NOD_Rate'] - min_value) / (max_value - min_value)

In [22]:
# drop the NOD_Rate column
A = A.drop(columns=['NOD_Rate'])

In [33]:
# reset the index
A = A.reset_index(drop=True)

In [34]:
A.head()

,STCNTY,percentile,NOD
0,01001,2,0.161264
1,01003,3,0.193679
2,01005,0,0.115073
3,01007,4,0.260130
4,01009,4,0.234198


In [35]:
type(A['STCNTY'][0])

str

In [26]:
#make STCNTY column int
A['STCNTY'] = A['STCNTY'].astype(str)

In [11]:
#make STCNTY column int
# A['STCNTY'] = A['STCNTY'].astype(int)

In [36]:
A

,STCNTY,percentile,NOD
0,01001,2,0.161264
1,01003,3,0.193679
2,01005,0,0.115073
3,01007,4,0.260130
4,01009,4,0.234198
...,...,...,...
3135,56037,4,0.226094
3136,56039,0,0.147488
3137,56041,4,0.233387
3138,56043,4,0.230956


In [37]:
# save A as a csv file
A.to_csv('/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/annotation 2018/annotation_NOD.csv', index=False)

In [6]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import pickle as pickle
from pylab import *
import warnings
import os

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# imoporting SVI data for the entire US(county level) 
us_svi = gpd.read_file('/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/SVI2018 census tracts with death rate HepVu-5 classes/VT/VT.shp')

In [7]:
# get the us_svi column names to a list
us_svi_columns = us_svi.columns.tolist()


In [8]:
us_svi_columns

['ST',
 'STATE',
 'ST_ABBR',
 'STCNTY',
 'COUNTY',
 'FIPS',
 'LOCATION',
 'AREA_SQMI',
 'E_TOTPOP',
 'M_TOTPOP',
 'E_HU',
 'M_HU',
 'E_HH',
 'M_HH',
 'E_POV',
 'M_POV',
 'E_UNEMP',
 'M_UNEMP',
 'E_PCI',
 'M_PCI',
 'E_NOHSDP',
 'M_NOHSDP',
 'E_AGE65',
 'M_AGE65',
 'E_AGE17',
 'M_AGE17',
 'E_DISABL',
 'M_DISABL',
 'E_SNGPNT',
 'M_SNGPNT',
 'E_MINRTY',
 'M_MINRTY',
 'E_LIMENG',
 'M_LIMENG',
 'E_MUNIT',
 'M_MUNIT',
 'E_MOBILE',
 'M_MOBILE',
 'E_CROWD',
 'M_CROWD',
 'E_NOVEH',
 'M_NOVEH',
 'E_GROUPQ',
 'M_GROUPQ',
 'EP_POV',
 'MP_POV',
 'EP_UNEMP',
 'MP_UNEMP',
 'EP_PCI',
 'MP_PCI',
 'EP_NOHSDP',
 'MP_NOHSDP',
 'EP_AGE65',
 'MP_AGE65',
 'EP_AGE17',
 'MP_AGE17',
 'EP_DISABL',
 'MP_DISABL',
 'EP_SNGPNT',
 'MP_SNGPNT',
 'EP_MINRTY',
 'MP_MINRTY',
 'EP_LIMENG',
 'MP_LIMENG',
 'EP_MUNIT',
 'MP_MUNIT',
 'EP_MOBILE',
 'MP_MOBILE',
 'EP_CROWD',
 'MP_CROWD',
 'EP_NOVEH',
 'MP_NOVEH',
 'EP_GROUPQ',
 'MP_GROUPQ',
 'EPL_POV',
 'EPL_UNEMP',
 'EPL_PCI',
 'EPL_NOHSDP',
 'SPL_THEME1',
 'RPL_THEME1',
 'EPL_